# Request Pattern Simulation

In [2]:
import heapq
import numpy as np
import datetime
import pandas as pd

# Define the lambda (arrival rate) for each hour of the day (weekday vs weekend), based on 90 days historical data
def arrival_rate(hour, weekday):
    if weekday < 5: 
        if hour == 0:
            return 1.923077
        elif hour == 1:
            return 1.953846
        elif hour == 2:
            return 5.707692
        elif hour == 3:
            return 2.584615
        elif hour == 4:
            return 21.138462
        elif hour == 5:
            return 3.230769
        elif hour == 6:
            return 3.276923
        elif hour == 7:
            return 2.907692
        elif hour == 8:
            return 26.446154
        elif hour == 9:
            return 49.953846
        elif hour == 10:
            return 38.646154
        elif hour == 11:
            return 20.892308
        elif hour == 12:
            return 17.276923
        elif hour == 13:
            return 29.984615
        elif hour == 14:
            return 24.200000
        elif hour == 15:
            return 11.215385
        elif hour == 16:
            return 9.461538
        elif hour == 17:
            return 9.569231
        elif hour == 18:
            return 10.338462
        elif hour == 19:
            return 13.523077
        elif hour == 20:
            return 9.015385
        elif hour == 21:
            return 6.015385
        elif hour == 22:
            return 4.815385
        else:
            return 7.738462
    else:
        if hour == 0:
            return 1.96
        elif hour == 1:
            return 1.08
        elif hour == 2:
            return 4.20
        elif hour == 3:
            return 2.40
        elif hour == 4:
            return 19.12
        elif hour == 5:
            return 1.28
        elif hour == 6:
            return 2.60
        elif hour == 7:
            return 3.20
        elif hour == 8:
            return 11.84
        elif hour == 9:
            return 52.24
        elif hour == 10:
            return 25.04
        elif hour == 11:
            return 15.48
        elif hour == 12:
            return 8.28
        elif hour == 13:
            return 16.04
        elif hour == 14:
            return 22.24
        elif hour == 15:
            return 5.44
        elif hour == 16:
            return 7.92
        elif hour == 17:
            return 3.72
        elif hour == 18:
            return 14.48
        elif hour == 19:
            return 10.60
        elif hour == 20:
            return 5.92
        elif hour == 21:
            return 5.16
        elif hour == 22:
            return 4.04
        else:
            return 8.60

# Define the start and end times for the simulation
start_time = datetime.datetime(2023, 4, 25, 0, 0, 0)
end_time = datetime.datetime(2023, 4, 25, 23, 59, 59)

# Define the list of wards, number of scans and their respective probabilities, based on 90 days historical data
ward_list = ['A1', 'A10', 'A11', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9',
       'ALG', 'B10', 'B2', 'B5', 'B6', 'B8', 'B9', 'C10', 'C11', 'C2', 'C3',
       'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'CM5', 'D5', 'D6', 'DGEN', 'E10',
       'E11', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'F10', 'F8',
       'G1', 'G10', 'G11', 'G2', 'G3', 'G4', 'G6', 'G7', 'G8', 'G9', 'H10',
       'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'KETM', 'M5', 'R1', 'R10',
       'R2', 'R7', 'R8', 'R9']
ward_probs = [0.013305110371938312, 0.010281221651043242, 0.0029230924301985686, 0.0018815307596680441,
              0.011557974666532272, 0.021200819809830997, 0.02929812182911669, 0.03296038705775627,
              0.023552733259416054, 0.02818936263145516, 0.003964654100729093, 0.006484561368141652,
              0.011692369720794274, 0.011692369720794274, 0.0022847159224540538, 0.02671101703457313,
              0.00047038268991701104, 0.00383025904646709, 0.008802876054161206, 0.008399690891375197,
              0.00010079629069650237, 0.013607499244027819, 0.02933172059268219, 0.029432516883378692,
              0.026139838053959613, 0.031146053825219233, 0.03111245506165373, 0.02509827638342909,
              3.359876356550079e-05, 0.00023519134495850552, 0.027382992305883144, 6.719752713100158e-05,
              0.00023519134495850552, 0.008366092127809697, 0.002889493666633068, 0.012868326445586803,
              0.018949702650942445, 0.02671101703457313, 0.031381245170177735, 0.02593824547256661,
              0.020596042065651982, 0.010885999395222256, 0.019823270503645466, 0.020730437119913986,
              0.0018143332325370426, 0.04304001612740651, 0.011255585794442765, 0.006114974968921143,
              0.010449215468870746, 0.013237912844807312, 0.020293653193562477, 0.04146087423982797,
              0.012901925209152303, 0.017672949635453416, 0.014749857205254846, 0.0003359876356550079,
              0.012935523972717805, 0.016228202802136882, 0.0037630615193360883, 0.007996505728589188,
              0.036253065887175355, 0.023754325840809058, 3.359876356550079e-05, 0.0003359876356550079,
              0.00010079629069650237, 3.359876356550079e-05, 0.00010079629069650237, 0.016698585492053893,
              0.015153042368040857, 0.0007391727984410174]
scan_options = [1,2,3,4,5,6,7,8]
scan_probs = [0.9363303936330394, 0.061733706173370616, 0.0012547501254750125, 
              0.00032265003226500323, 0.00014340001434000142, 0.00010755001075500108, 
              7.170000717000071e-05, 3.5850003585000356e-05]

# Initialize the priority queue with the first arrival
event_queue = [(start_time, 'arrival')]

# Generate a list of x-ray request arrivals
xray_arrivals = []
while event_queue:
    # Get the next event from the priority queue
    event_time, event_type = heapq.heappop(event_queue)

    # If the event time is after the end time, stop the simulation
    if event_time > end_time:
        break

    # If the event is an arrival, generate a new arrival and schedule the next one
    if event_type == 'arrival':
        hour = event_time.hour
        day_of_week = event_time.weekday()
        mean_interarrival_time = 60 / arrival_rate(hour, day_of_week)
        ward = np.random.choice(ward_list, p=ward_probs)
        num_scans = np.random.choice(scan_options, p=scan_probs)
        #is_urgent = np.random.choice([True, False], p=[0.15, 0.85])
        xray_arrivals.append((event_time, ward, num_scans))
        #xray_arrivals.append((event_time, ward, num_scans, is_urgent))
        interarrival_time = np.random.exponential(scale=mean_interarrival_time)
        next_arrival_time = event_time + datetime.timedelta(minutes=interarrival_time)
        heapq.heappush(event_queue, (next_arrival_time, 'arrival'))

# Full version: Convert the xray_arrivals list to a pandas DataFrame
xray_df = pd.DataFrame(xray_arrivals, columns=['Arrival Time', 'Ward', 'Number of Scans'])
#xray_df = pd.DataFrame(xray_arrivals, columns=['Arrival Time', 'Ward', 'Number of Scans', "Is_Urgent"])

# Aggregate version: Group the xray_df DataFrame by ward and sum the "Num Scans" column for each group
ward_scans_df = xray_df.groupby('Ward')['Number of Scans'].sum().reset_index()
#ward_scans_df = xray_df.groupby('Ward')[['Number of Scans', "Is_Urgent"]].sum().reset_index()
       
# Print the number of x-ray request arrivals generated
print(f"Generated {len(xray_arrivals)} x-ray request arrivals.")

Generated 302 x-ray request arrivals.


In [76]:
xray_df

,Arrival Time,Ward,Number of Scans
0,2023-04-25 00:00:00.000000,A8,1
1,2023-04-25 00:03:41.611312,C8,1
2,2023-04-25 00:08:58.216467,C9,1
3,2023-04-25 01:17:35.435393,C7,1
4,2023-04-25 01:25:09.386007,H3,1
...,...,...,...
301,2023-04-25 21:52:46.884541,C4,1
302,2023-04-25 22:01:32.716380,A10,1
303,2023-04-25 22:27:59.279500,G11,1
304,2023-04-25 22:41:44.534615,A9,1


In [77]:
ward_scans_df

,Ward,Number of Scans
0,A1,5
1,A10,2
2,A3,5
3,A4,9
4,A5,9
5,A6,8
6,A7,4
7,A8,8
8,A9,1
9,ALG,1


# To function (same as above but in a function)

In [3]:
# Define the lambda (arrival rate) for each hour of the day (weekday vs weekend), based on 90 days historical data
def arrival_rate(hour, weekday):
    if weekday < 5: 
        if hour == 0:
            return 1.923077
        elif hour == 1:
            return 1.953846
        elif hour == 2:
            return 5.707692
        elif hour == 3:
            return 2.584615
        elif hour == 4:
            return 21.138462
        elif hour == 5:
            return 3.230769
        elif hour == 6:
            return 3.276923
        elif hour == 7:
            return 2.907692
        elif hour == 8:
            return 26.446154
        elif hour == 9:
            return 49.953846
        elif hour == 10:
            return 38.646154
        elif hour == 11:
            return 20.892308
        elif hour == 12:
            return 17.276923
        elif hour == 13:
            return 29.984615
        elif hour == 14:
            return 24.200000
        elif hour == 15:
            return 11.215385
        elif hour == 16:
            return 9.461538
        elif hour == 17:
            return 9.569231
        elif hour == 18:
            return 10.338462
        elif hour == 19:
            return 13.523077
        elif hour == 20:
            return 9.015385
        elif hour == 21:
            return 6.015385
        elif hour == 22:
            return 4.815385
        else:
            return 7.738462
    else:
        if hour == 0:
            return 1.96
        elif hour == 1:
            return 1.08
        elif hour == 2:
            return 4.20
        elif hour == 3:
            return 2.40
        elif hour == 4:
            return 19.12
        elif hour == 5:
            return 1.28
        elif hour == 6:
            return 2.60
        elif hour == 7:
            return 3.20
        elif hour == 8:
            return 11.84
        elif hour == 9:
            return 52.24
        elif hour == 10:
            return 25.04
        elif hour == 11:
            return 15.48
        elif hour == 12:
            return 8.28
        elif hour == 13:
            return 16.04
        elif hour == 14:
            return 22.24
        elif hour == 15:
            return 5.44
        elif hour == 16:
            return 7.92
        elif hour == 17:
            return 3.72
        elif hour == 18:
            return 14.48
        elif hour == 19:
            return 10.60
        elif hour == 20:
            return 5.92
        elif hour == 21:
            return 5.16
        elif hour == 22:
            return 4.04
        else:
            return 8.60

In [79]:
#start_time: the start time of the simulation
#end_time: the end time of the simulation
#arrival_rate: a function that takes in an hour and day of the week and returns the arrival rate for that hour and day
#ward_list: a list of ward names
#ward_probs: a list of probabilities for each ward
#scan_options: a list of scan options
#scan_probs: a list of probabilities for each scan option

def simulate_xray_requests(start_time, end_time, arrival_rate, ward_list, ward_probs, scan_options, scan_probs):
    
    # Initialize the priority queue with the first arrival
    event_queue = [(start_time, 'arrival')]

    # Generate a list of x-ray request arrivals
    xray_arrivals = []
    while event_queue:
        # Get the next event from the priority queue
        event_time, event_type = heapq.heappop(event_queue)

        # If the event time is after the end time, stop the simulation
        if event_time > end_time:
            break

        # If the event is an arrival, generate a new arrival and schedule the next one
        if event_type == 'arrival':
            hour = event_time.hour
            day_of_week = event_time.weekday()
            mean_interarrival_time = 60 / arrival_rate(hour, day_of_week)
            ward = np.random.choice(ward_list, p=ward_probs)
            num_scans = np.random.choice(scan_options, p=scan_probs)
            #is_urgent = np.random.choice([True, False], p=[0.05, 0.95])
            xray_arrivals.append((event_time, ward, num_scans))
            #xray_arrivals.append((event_time, ward, num_scans, is_urgent))
            interarrival_time = np.random.exponential(scale=mean_interarrival_time)
            next_arrival_time = event_time + datetime.timedelta(minutes=interarrival_time)
            heapq.heappush(event_queue, (next_arrival_time, 'arrival'))
    
    # Full version: Convert the xray_arrivals list to a pandas DataFrame
    xray_df = pd.DataFrame(xray_arrivals, columns=['Exam_Datetime', 'Ward', 'Number of Scans'])
    #xray_df = pd.DataFrame(xray_arrivals, columns=['Arrival Time', 'Ward', 'Number of Scans', "Is_Urgent"])

    # Aggregate version: Group the xray_df DataFrame by ward and sum the "Num Scans" column for each group
    ward_scans_df = xray_df.groupby('Ward')['Number of Scans'].sum().reset_index()
    #ward_scans_df = xray_df.groupby('Ward')[['Number of Scans', "Is_Urgent"]].sum().reset_index()

    return xray_df, ward_scans_df

In [80]:
# Define the list of wards, number of scans and their respective probabilities, based on 90 days historical data
ward_list = ['A1', 'A10', 'A11', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9',
       'ALG', 'B10', 'B2', 'B5', 'B6', 'B8', 'B9', 'C10', 'C11', 'C2', 'C3',
       'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'D5', 'D6', 'E10', 'E11', 'E2',
       'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'F10', 'F8', 'G1', 'G10',
       'G11', 'G2', 'G3', 'G4', 'G6', 'G7', 'G8', 'G9', 'H10', 'H2', 'H3',
       'H4', 'H5', 'H6', 'H7', 'H8', 'M5', 'R1', 'R10', 'R2', 'R7', 'R8',
       'R9']
ward_probs = [0.013306898753318324,
 0.010282603582109614,
 0.0029234853321684195,
 0.0018817836620854196,
 0.011559528209953292,
 0.021203669478141066,
 0.02930205988104439,
 0.03296481736617494,
 0.023555899055747842,
 0.028193151651601196,
 0.00396518700225142,
 0.006485432978258678,
 0.011693941328673678,
 0.011693941328673678,
 0.002285023018246581,
 0.02671460734567694,
 0.0004704459155213549,
 0.0038307738835310326,
 0.008804059276185355,
 0.008400819920024194,
 0.00010080983904029033,
 0.013609328270439194,
 0.029335663160724487,
 0.029436472999764776,
 0.026143351591115294,
 0.031150240263449713,
 0.031116636983769617,
 0.025101649921032292,
 0.00023522295776067745,
 0.027386672939278874,
 0.00023522295776067745,
 0.008367216640344097,
 0.002889882052488323,
 0.012870056117477066,
 0.018952249739574583,
 0.02671460734567694,
 0.03138546322121039,
 0.025941731913034713,
 0.020598810443899326,
 0.010887462616351356,
 0.0198259350112571,
 0.020733223562619713,
 0.001814577102725226,
 0.04304580127020397,
 0.01125709869283242,
 0.006115796901777613,
 0.010450619980510098,
 0.01323969219395813,
 0.020296380926778452,
 0.041466447125239426,
 0.012903659397157163,
 0.017675325111730904,
 0.014751839779562485,
 0.00033603279680096776,
 0.01293726267683726,
 0.016230384085486743,
 0.0037635673241708393,
 0.007997580563863033,
 0.03625793877482442,
 0.02375751873382842,
 0.00033603279680096776,
 0.00010080983904029033,
 3.360327968009678e-05,
 0.00010080983904029033,
 0.0167008300010081,
 0.015155079135723646,
 0.0007392721529621291]
scan_options = [1,2,3,4,5,6,7,8]
scan_probs = [0.9363303936330394, 0.061733706173370616, 0.0012547501254750125, 
              0.00032265003226500323, 0.00014340001434000142, 0.00010755001075500108, 
              7.170000717000071e-05, 3.5850003585000356e-05]

In [81]:
simulate_xray_requests(start_time, end_time, arrival_rate, ward_list, ward_probs, scan_options, scan_probs)

(                 Exam_Datetime Ward  Number of Scans
 0   2023-04-25 00:00:00.000000   H8                1
 1   2023-04-25 00:32:09.670284   E5                1
 2   2023-04-25 00:54:51.269149   A5                2
 3   2023-04-25 01:15:02.286977   C5                1
 4   2023-04-25 01:22:59.662622   R7                1
 ..                         ...  ...              ...
 301 2023-04-25 23:17:13.075207   R8                1
 302 2023-04-25 23:18:50.113318   A5                1
 303 2023-04-25 23:21:47.662292   F8                1
 304 2023-04-25 23:45:26.373809  F10                1
 305 2023-04-25 23:50:31.292084   A8                1
 
 [306 rows x 3 columns],
    Ward  Number of Scans
 0    A1                3
 1   A10                4
 2    A2                2
 3    A3                3
 4    A4                7
 5    A5                8
 6    A6               20
 7    A7                8
 8    A8               12
 9    A9                1
 10  ALG                4
 11  B10     

In [84]:
from pathlib import Path

# Define the hour pair and minute pair for each time period
hour_pair = [(23, 10), (10, 10), (10, 11), (11, 11), (11, 14), (14, 14), (14, 15), (15, 15), 
            (15, 19), (19, 19), (19, 20), (20, 20), (20, 22), (22, 22), (22, 23),(23, 23)]
minute_pair = [(30, 0), (0, 30), (30, 0), (0, 30), (30, 0), (0, 30),(30, 0), (0, 30),
             (30, 0), (0, 30), (30, 0), (0, 30), (30, 0), (0, 30), (30, 0), (0, 30)]


# Loop through the time periods and simulate the x-ray requests
# generate 5 sets of 30-day data
for loop_time in range(1, 6):
    for day in range(1, 31):
        midnight = 1
        for hours, minutes in zip(hour_pair, minute_pair):
            if midnight == 1:
                start_time = datetime.datetime(2023, 5, day, hours[0], minutes[0], 0)
                end_time = datetime.datetime(2023, 5, day+1, hours[1], minutes[1], 0)
                filename = 'day ' + str(day-1) + " " + str(hours[0]) + "_" + str(minutes[0]) + ' to day ' + str(day) + " " + str(hours[1]) + "_" + str(minutes[1])
                midnight = 0
            else:
                start_time = datetime.datetime(2023, 5, day+1, hours[0], minutes[0], 0)
                end_time = datetime.datetime(2023, 5, day+1, hours[1], minutes[1], 0)
                filename = 'day ' + str(day) + " " + str(hours[0]) + "_" + str(minutes[0]) + ' to day ' + str(day) + " " + str(hours[1]) + "_" + str(minutes[1])

            # simulate the x-ray requests
            myfile = simulate_xray_requests(start_time, end_time, arrival_rate, ward_list, ward_probs, scan_options, scan_probs)[-1]
            # save the data to csv
            output_dir = Path(f"30day_data_{loop_time}")
            output_dir.mkdir(exist_ok=True, parents=True)
            myfile.to_csv(f"30day_data_{loop_time}/{filename}.csv", index=False)


# To CSV File

In [83]:
# import csv

# # Define the number of times you want to loop
# num_files_to_create = 30

# # Loop through the function and create a new CSV file for each iteration
# for i in range(num_files_to_create):
    
#     # Call the function to create the contents of the CSV file
#     my_dataframe_full, my_dataframe_agg = simulate_xray_requests(start_time, end_time, arrival_rate, ward_list, ward_probs, scan_options, scan_probs)
    
#     # Create a new CSV file with a unique name
#     my_dataframe_full.to_csv(f"simulated_full_24hrs/my_csv_file_full_24hrs_{i+1}.csv", index=False)
#     my_dataframe_agg.to_csv(f"simulated_agg_24hrs/my_csv_file_agg_24hrs_{i+1}.csv", index=False)

OSError: Cannot save file into a non-existent directory: 'simulated_full_24hrs'